In [1]:
import os
import re
import json
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
%matplotlib inline

### The class MoleculeRot is defined. 
It creates an object for each molecule with a variety of properties including a dictionary (and a normalized dictionary) for it's energies at each dihedral rotation. Each object also has descriptive properites such as name, smile ring number, unit number, polymer nuber, etc. There are only a couple class funtions, but these draw the molecule structure and plot the PE curve. 

In [8]:
class MoleculeRot:
    def __init__(
        self, 
        name,
        master_dir,
        energy_fn = "master_energy.json", 
        homo_fn = "master_homos.json", 
        lumo_fn = "master_lumos.json", 
        unconst_fn = "master_geomopt.json",
        omega_path = "master_omega.json",
        smiles_fn = "master_smiles.json"      
    ):
        self.origin_fn = os.path.abspath(origin_fn)
        self.smi_dir = os.path.abspath(smi_dir)
        self.unconst_file = os.path.abspath(unconst_file)
        
        self.ring_num = int(self.name.split('_')[1])
        self.unit_num = int(self.name.split('_')[2])
        self.polymer_num = int(self.name.split('_')[3])
        self.substituents = self.name.split('_')[4]
        
        self.energy_path = os.path.join(master_dir,self.energy_fn)
        self.homo_path = os.path.join(master_dir,self.homo_fn)
        self.lumo_path = os.path.join(master_dir,self.lumo_fn)
        self.unconst_path = os.path.join(master_dir,self.unconst_fn)
        self.omega_path = os.path.join(master_dir,self.smiles_fn)
        self.smiles_path = os.path.join(master_dir,self.omega_fn)
        
        self.energy_dict = get_data_from_master(self.energy_path)
        self.homo_dict = get_data_from_master(self.homo_path)
        self.lumo_dict = get_data_from_master(self.lumo_path)
        self.unconst_data = get_data_from_master(self.unconst_path)
        self.tuned_omega = get_data_from_master(self.omega_path)
        self.smiles = get_data_from_master(self.smiles_path)
     
    def __str__(self):
        return  f'name: {self.name}\n{self.ring_num} ring type, {self.unit_num} monomer units, {self.substituents} substituents\nenergy dictionary: {self.energy_dict}'
    
    @classmethod
    def get_data_from_master(master_file):
        with open(master_file, 'r') as fn: 
            _dict = json.load(fn)
        try: 
            _data = _dict[self.name]
            return _data
        except KeyError: 
            try: 
                _data = [v for k,v in _dict.items() if k.startswith(self.name)][0]
                return _unconst_data
            except IndexError:  
                return None
    
    @property
    def unconst_energy(self): 
        _dict = self.unconst_data
        if _dict is None:
            return None
        else: 
            return _dict[1] / 27.2114
        
    @property
    def unconst_angle(self): 
        _dict = self.unconst_data
        if _dict is None:
            return None
        else:
            return abs(self.unconst_data[0])
        
    @property
    def min_e(self):
        return min(self.energy_dict.values())
    
    @property
    def norm_energy_dict(self):
        if self.unconst_energy is None: 
            _norm_edict = {deg: 627.5*(eng - self.min_e) for deg, eng in self.energy_dict.items()}
            _sorted_norm_edict = dict(sorted(_norm_edict.items()))
            return _sorted_norm_edict 
        else: 
            _norm_edict = {deg: 627.5*(eng - self.unconst_energy) for deg, eng in self.energy_dict.items()}
            _sorted_norm_edict = dict(sorted(_norm_edict.items()))
            return _sorted_norm_edict 
    
    
    def draw_structure(self, out_dir=None):
        molecule = Chem.MolFromSmiles(self.smile)
        AllChem.Compute2DCoords(molecule)
        if out_dir is not None: 
            Draw.MolToFile(molecule, out_dir+'img_{}.png'.format(self.name))
        return Draw.MolToImage(molecule)
    
    def plot_pe_curve(self, title, out_dir):
        """
        Plots the energy vs the torsion angle for a molecule given a torsion energy
        file. Saves the plot to a png file.
        """
        if self.unconst_data is None:
            print("No unconstrained dictionary found for {}.".format(self.name))
        phi, energy = self.norm_energy_dict.keys(), self.norm_energy_dict.values()
        min_energy, max_energy = min(energy), max(energy)

        fig, ax = plt.subplots()
        ax.set_xlim(0, 3)
        ax.set_ylim(0, 3)
        
        plt.scatter(phi, energy)
        plt.xlim(min(phi)-3, max(phi)+3)
        plt.xticks(np.linspace(start=0, stop=180, num=7))
        plt.ylim(top = 15, bottom = -1)
        plt.yticks(np.linspace(start=0, stop=14, num=8))
        plt.xlabel("dihedral angle (degrees)")
        plt.ylabel("energy (kcal/mol)")
        plt.title(title)
        fig.set_facecolor('w')
        plt.savefig(out_dir+'torsionE_plt_{}.png'.format(title), dpi=300, bbox_inches='tight')
        
        
#         image = self.Draw()
#         image.set_size_inches(1.5, 1.5)
#         ab = AnnotationBbox(image, (10, -10))
#         fig.add_artist(image)
        plt.close('all')
    
    def get_angle_from_energy(self, energy):
        for angle, eg in self.energy_dict.items():
             if energy == eg:
                return angle
        return "Dihedral angle doesn't exist for {}.".format(energy)
    
    @staticmethod
    def write_json(data, filename):
        with open(filename,'w') as f:
            json.dump(data, f, indent=2)

    
    def write_nnff_json(self, json_outdir): 
        for angle in self.norm_energy_dict.keys(): 
            json_data = {
                "molecule_name" : self.name,
                "degree" : angle,
                "smile" : self.smile,
                "energy" : self.norm_energy_dict[angle]
            }
            self.write_json(json_data, "{}/{}_{}deg.json".format(json_outdir,self.name,int(angle)))


In [11]:
def make_all_mol_list(master_dir):
    all_molecules = []
    master_files = [x for x in os.listdir(master_dir) if x.startswith('master')]
    master_path = os.path.join(master_dir,master_files[0])
    with open(master_path, 'r') as fn: 
        data = json.load(fn)
    mol_names = data.keys()
    for name in mol_names:
        try: 
            mol = MoleculeRot(name, master_dir) 
            all_molecules.append(mol)
        except: 
            print("Error. Did not convert {} to Molecule Object.".format(f))
    return all_molecules

In [12]:
home = os.getcwd()
master_dir = os.path.abspath(os.path.join(home,'masters/'))

all_molecules = make_all_mol_list(master_dir)

for mol in all_molecules: 
    print(mol)

NameError: name 'f' is not defined

In [47]:
home = os.getcwd()
energy_dir = os.path.abspath(os.path.join(home,'energies/'))
smi_dir = os.path.abspath(os.path.join(home,'smiles/'))
unconst_file = os.path.join(home,'geomopt_jsons/master_geomopt.json')

all_molecules = make_all_mol_list(energy_dir,smi_dir, unconst_file)
dic = {}
for mol in all_molecules: 
    dic[mol.name] = mol.get_angle_from_energy(mol.min_e)

    
# # Find molecules where the min energy is significantly lower than the unconstrained opt energy for redo. 
mol.write_json(dic,os.path.join(home,'lowest_e_angles.txt'))
# a = 0
# b = 0
# redo_list = []
# for mol in all_molecules: 
#     min_energy = mol.min_e
#     geomopt_energy = mol.unconst_energy
#     if geomopt_energy is not None:
#         if min_energy < geomopt_energy:
#             min_angle = mol.get_angle_from_energy(min_energy)
#             geomopt_angle = mol.unconst_angle
#             a += 1
#             if min_angle <= (geomopt_angle+10) and min_angle >= (geomopt_angle-10): 
#                 pass
#             else:
#                 print(mol.name)
#                 print("Minimum Energy Angle:\t {}".format(min_angle))
#                 print("Geomopt Energy Angle:\t {}".format(geomopt_angle))
#                 print("-------------------------------------")
#                 redo_list.append(mol.name)
#                 b += 1
# print(a)
# print(b)

No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_7_06_c-n-CF found.
No unconstrained data for mols_65_7_06_c-n-CF found.
No unconstrained data for mols_65_7_10_c-cF-CF found.
No unconstrained data for mols_65_7_10_c-cF-CF found.
No unconstrained data for mols_65_7_16_n-n-C found.
No unconstrained data for mols_65_7_16_n-n-C found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_29_cF-cF-N found.
No unconstrained data for mols_65_7_29_cF-cF-N found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.


### Make molecule object list
This function makes a list of MoleculeRot objects for all molecules in the energy_dir.

In [14]:
def make_all_mol_list(energy_dir, smi_dir, unconst_file):
    all_molecules = []
    for f in os.listdir(energy_dir):
        fpath = os.path.join(energy_dir,f)
        if f.startswith("energies"):
            try: 
                mol = MoleculeRot(fpath, smi_dir, unconst_file)
                all_molecules.append(mol)
            except: 
                print("Error. Did not convert {} to Molecule Object.".format(f))
    return all_molecules

### Parse small list
This function parses a list of molecules with specifified properties from a list of MoleculeRot objects.

In [15]:
def make_small_mol_list(all_molecules,ring_num=None,unit_num=None,polymer_num=None,substituents=None):
    mol_list = []
    for mol in all_molecules:
        if ring_num is not None:
            if mol.ring_num != ring_num:
                continue
        if unit_num is not None:
            if mol.unit_num != unit_num:
                continue
        if polymer_num is not None:
            if mol.polymer_num != polymer_num:
                continue
        if substituents is not None:
            if bool(re.search(substituents, mol.substituents)) == False:
                continue
        mol_list.append(mol)
    return mol_list

### Plot overlay plot
This function plots an overlayed plot of all molecule PE cureves in the mol_list

In [16]:
def overlay_plot(mol_list,title,out_dir,varying_atrib='unit_num',legend_outside=True,draw_1unit=False):
    """
    Makes overlay plot of allenergies files in cwd.
    """       
    fig, ax = plt.subplots()
    for mol in mol_list: 
        if mol.unit_num == 1: 
            mol_image = mpimg.pil_to_array(mol.draw_structure())
        try: 
            phi, energy = mol.norm_energy_dict.keys(), mol.norm_energy_dict.values()
            ax.scatter(phi, energy, label = eval('mol.'+varying_atrib))
            ax.plot(phi, energy)
        except: 
            print('Error. Could not plot {} {} for {}.'.format(varying_atrib,eval('mol.'+varying_atrib),title))
    
    ax.set_xlim(-3, 183)
    ax.set_xticks(np.linspace(start=0, stop=180, num=7))
    ax.set_ylim(top = 15, bottom = -1)
    ax.set_yticks(np.linspace(start=0, stop=14, num=8))
    ax.set_xlabel("dihedral angle (degrees)")
    ax.set_ylabel("energy (kcal/mol)")
    ax.set_title(title)
    if draw_1unit == True:
        newax = fig.add_axes([0.7, .2, 0.4, 0.4], anchor='NE')
        img = newax.add_artist(AnnotationBbox(OffsetImage(mol_image,zoom=0.5),(1,1)))
        newax.axis('off')
    if legend_outside == True: 
        fig.legend(loc='lower left',bbox_to_anchor=(1, 0))
    fig.patch.set_facecolor("w")
    fig.savefig(out_dir+'torsionE_OverlayPlt_{}.png'.format(title), dpi=300, bbox_inches='tight',bbox_extra_artists=(img,))
    plt.close('all')

### Plot average plot
This function plots the average PE curve (with error bars) of all molecules objects in mol_list

In [17]:
def average_plot(mol_list,title,out_dir):
    """
    Makes overlay plot of polymer unit energy averages (with error bars) in cwd.
    """
    data = np.zeros([19,3])
    for n,d in enumerate(np.linspace(0,180,19)):
        data[n][0] = d
        energies = []
        for mol in mol_list: 
            phi, energy = mol.norm_energy_dict.keys(), mol.norm_energy_dict.values()
            try:   
                energies.append(mol.norm_energy_dict[d])
                np_energies = np.array(energies)
                data[n][1] = np.average(np_energies)
                data[n][2] = np.std(np_energies)/np.sqrt(len(np_energies))
            except: 
                print('Error. Did not find energy for {} at {} degree rotation.'.format(mol.name,d))
    min_energy = np.amin(data[:,1]) - np.amax(data[:,2])
    max_energy = np.amax(data[:,1]) + np.amax(data[:,2])
    energy_range = max_energy - min_energy
    
    print('\n\nNumber of molecules averaged: {}'.format(len(mol_list)))
    fig = plt.figure()
    plt.scatter(data[:,0], data[:,1], label = title)
    plt.errorbar(data[:,0], data[:,1], yerr=data[:,2], linestyle="None")
    plt.xlim(min(phi)-3, max(phi)+3)
    plt.xticks(np.linspace(start=0, stop=180, num=7))
    plt.ylim(top = 15, bottom = -1)
    plt.yticks(np.linspace(start=0, stop=14, num=8))
    plt.xlabel("dihedral angle (degrees)")
    plt.ylabel("energy (kcal/mol)")
    plt.title(title)
    plt.savefig(out_dir+'torsionE_AvgPlt_{}.png'.format(title), dpi=300, bbox_inches='tight')
#     plt.savefig(out_dir+'torsionE_AvgPlt_byUnitNum.png'.format(title), dpi=300, bbox_inches='tight')
    plt.close('all')

# Make General Plots
Establish file paths and list of all molecule objects. Run before making any plots. 

In [18]:
home = os.getcwd()
energy_dir = os.path.abspath(os.path.join(home,'energies/'))
smi_dir = os.path.abspath(os.path.join(home,'smiles/'))
unconst_file = os.path.join(home,'geomopt_jsons/master_geomopt.json')

all_molecules = make_all_mol_list(energy_dir,smi_dir, unconst_file)
print("Done making all_molecules list.")  

Done making all_molecules list.


### Make plot for each unit of each polymer

In [39]:
out_dir = os.path.join(home,'plots/')
for mol in all_molecules:
#     mol.plot_pe_curve(mol.name,out_dir)
    mol.write_nnff_json(os.path.join(home,'jsons/'))
    print('Done plotting {}'.format(mol.name))


Done plotting mols_66_1_11_n-n-cF-cF
Done plotting mols_66_1_12_c-c-cF-cOC
Done plotting mols_66_1_13_c-n-cF-cOC
Done plotting mols_66_1_14_n-n-cF-cOC
Done plotting mols_66_1_15_c-c-cOC-cOC
Done plotting mols_66_1_16_c-n-cOC-cOC
Done plotting mols_66_1_17_n-n-cOC-cOC
Done plotting mols_66_3_00_c-c-c-
Done plotting mols_66_3_01_c-n-c-
Done plotting mols_66_3_02_n-n-c-
Done plotting mols_66_3_03_c-c-c-cF
Done plotting mols_66_3_04_c-n-c-cF
Done plotting mols_66_3_05_n-n-c-cF
Done plotting mols_66_3_06_c-c-c-cOC
Done plotting mols_66_3_07_c-n-c-cOC
Done plotting mols_66_3_08_n-n-c-cOC
Done plotting mols_66_3_09_c-c-cF-cF
Done plotting mols_66_3_10_c-n-cF-cF
Done plotting mols_66_3_11_n-n-cF-cF
Done plotting mols_66_3_12_c-c-cF-cOC
Done plotting mols_66_3_13_c-n-cF-cOC
Done plotting mols_66_3_14_n-n-cF-cOC
Done plotting mols_66_3_15_c-c-cOC-cOC
Done plotting mols_66_3_16_c-n-cOC-cOC
Done plotting mols_66_3_17_n-n-cOC-cOC
Done plotting mols_66_5_00_c-c-c-
Done plotting mols_66_5_01_c-n-c-
D

No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- 

No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- found.
No unconstrained data for mols_66_7_01_c-n-c- 

No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF

No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF found.
No unconstrained data for mols_65_5_34_cF-cOC-CF

...

No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconst

No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconstrained data for mols_65_7_32_cF-cOC-C found.
No unconst

No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N

No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N

No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
Done plotting mols_65_7_37_cOC-cOC-N
Done plotting mols_65_7_38_cOC-cOC-CF
Done plotting mols_66_1_00_c-c-c-
Done plotting mols_66_1_01_c-n-c-
Done plotting mols_66_1_02_n-n-c-
Done plotting mols_66_1_03_c-c-c-cF
Done plotting mols_66_1_04_c-n-c-cF
Done plotting mols_66_1_05_n-n-c-cF
Done plotting mols_66_1_06_c-c-c-cOC
Done plotting mols_66_1_07_c-n-c-cOC
Done plotting mols_66_1_08_n-n-c-cOC
Done plotting mols_66_1_09_c-c-cF-cF
Done plotting mols_66_1_10_c-n-cF-cF
Done plotting mols_55_1_00

### Make overlay plot for each polymer

In [40]:
out_dir = os.path.join(home,'plots_overlay/')
ring_type = [55,65,66]
for r in ring_type:
    i = 0
    while i <= 40:
        unit_overlay_list = make_small_mol_list(all_molecules,ring_num=r, polymer_num=i)
        if len(unit_overlay_list) != 0:
            try: 
                first_mol = unit_overlay_list[0]
                mol_name = "mol_{}__{}_{}".format(first_mol.ring_num, first_mol.polymer_num,first_mol.substituents)
                overlay_plot(unit_overlay_list,mol_name,out_dir,draw_1unit=True)
                print('Done plotting overlay for {}'.format(mol_name))
                i += 1
            except:
                i += 1
                print('Error plotting overlay for {} rings and {} polymer number'.format(r,i))
        else: 
            print('There are no molecules for {} rings and {} polymer number'.format(r,i))
            break 

Done plotting overlay for mol_55__0_C-C
Done plotting overlay for mol_55__1_C-CF
Done plotting overlay for mol_55__2_C-N
Done plotting overlay for mol_55__3_C-COC
Done plotting overlay for mol_55__4_CF-CF
Done plotting overlay for mol_55__5_CF-N
Done plotting overlay for mol_55__6_CF-COC
Done plotting overlay for mol_55__7_N-N
Done plotting overlay for mol_55__8_N-COC
Done plotting overlay for mol_55__9_COC-COC
There are no molecules for 55 rings and 10 polymer number
Done plotting overlay for mol_65__0_c-c-C
Done plotting overlay for mol_65__1_c-c-N
Done plotting overlay for mol_65__2_c-c-CF
Done plotting overlay for mol_65__3_c-c-COC
Done plotting overlay for mol_65__4_c-n-C
Done plotting overlay for mol_65__5_c-n-N
No unconstrained data for mols_65_7_06_c-n-CF found.
No unconstrained data for mols_65_7_06_c-n-CF found.
No unconstrained data for mols_65_7_06_c-n-CF found.
No unconstrained data for mols_65_7_06_c-n-CF found.
No unconstrained data for mols_65_7_06_c-n-CF found.
No unco

Done plotting overlay for mol_65__24_n-cOC-C
Done plotting overlay for mol_65__25_n-cOC-N
Done plotting overlay for mol_65__26_n-cOC-CF
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
No unconstrained data for mols_65_7_27_n-cOC-COC found.
Done plotting overlay for mol_65__27_n-cOC-COC
Done plotting overlay for mol_65__28_cF-cF-C
No unconstrained data for mols_65_7_29_cF-cF-N found.
No unconstrained data for mols_65_7_29_cF-cF-N

No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N found.
No unconstrained data for mols_65_7_37_cOC-cOC-N

### Make average plots for each unit size 

In [ ]:
out_dir = os.path.join(home,'plts_other/')
unit_nums = [1,3,5,7]
for u in unit_nums:
    try:
        unit_list = make_small_mol_list(all_molecules,unit_num=u)
        average_plot(unit_list, str(u)+'_subunits', out_dir)
        overlay_plot(unit_list,str(u)+'_subunits',out_dir,legend_outside=False)
        print('Done plotting average energies for {} units'.format(u))
    except:
        print('Error plotting average energies for {} units'.format(u))

# Specific plots

In [ ]:
# Make plots comparing molecules with the same cromophore ring structure
out_dir = os.path.join(home,'other_plts/')
ring_numbers = [55,65,66]
for ring in ring_numbers: 
    ring_overlay_list = make_small_mol_list(all_molecules,ring_num=ring)
    overlay_plot(ring_overlay_list,'ring{}_plt'.format(ring),out_dir,varying_atrib='substituents')
    average_plot(ring_overlay_list,'ring{}_plt'.format(ring),out_dir)


In [ ]:
 # Make plots comaring molecules with the same substituents
out_dir = os.path.join(home,'other_plts/')
all_substituents = set(())
print(all_substituents)
for mol in all_molecules: 
    all_substituents.add(mol.substituents)
for sub in all_substituents: 
    sub_overlay_list = make_small_mol_list(all_molecules,substituents=sub)
#     overlay_plot(sub_overlay_list,'{}_plt'.format(sub),os.path.join(out_dir,'substituents/'),varying_atrib='name')
    average_plot(sub_overlay_list,'{}_plt'.format(sub),os.path.join(out_dir,'substituents/'))

class MoleculeRot:"master_energy.json")
    write_master_json(json_dir, master_energy_file, prop="energies")

    master_homos_file = os.path.join(json_dir, "master_homos.json")
    write_master_json(json_dir, master_homos_file, prop="homos")

    master_lumos_file = os.path.join(json_dir, "master_lumos.json")
    write_master_json(json_dir, master_lumos_file, prop="lumos")
    def __init__(self, origin_fn, smi_dir, unconst_file):
        self.origin_fn = os.path.abspath(origin_fn)
        self.smi_dir = os.path.abspath(smi_dir)
        self.unconst_file = os.path.abspath(unconst_file)
        
        self.name = str(origin_fn.split('energies_')[-1]).strip('.cvs')
        self.ring_num = int(self.name.split('_')[1])
        self.unit_num = int(self.name.split('_')[2])
        self.polymer_num = int(self.name.split('_')[3])
        self.substituents = self.name.split('_')[4]
     
    def __str__(self):
        return  f'name: {self.name}\n{self.ring_num} ring type, {self.unit_num} monomer units, {self.substituents} substituents\nenergy dictionary: {self.energy_dict}'
    
    @property
    def unconst_data(self):
        with open(self.unconst_file, 'r') as unconst_fn: 
            _dict = json.load(unconst_fn)
        try: 
            _unconst_data = _dict[self.name]
            return _unconst_data
        except KeyError: 
            try: 
                _unconst_data = [v for k,v in _dict.items() if k.startswith(self.name)][0]
                return _unconst_data
            except IndexError:  
                print("No unconstrained data for {} found.".format(self.name))
                return None
    @property
    def unconst_energy(self): 
        _dict = self.unconst_data
        if _dict is None:
            return None
        else: 
            return _dict[1] / 27.2114
    @property
    def unconst_angle(self): 
        _dict = self.unconst_data
        if _dict is None:
            return None
        else:
            return abs(self.unconst_data[0])
        
    @property
    def min_e(self):
        return min(self.energy_dict.values())
    
    @property
    def energy_dict(self):
        data = dict(np.genfromtxt(fname=self.origin_fn,delimiter=',', dtype='float'))
        if self.unconst_energy == None: 
            return data
        else: 
            data[self.unconst_angle] = self.unconst_energy
            return data
    @property
    def norm_energy_dict(self):
        if self.unconst_energy is None: 
            _norm_edict = {deg: 627.5*(eng - self.min_e) for deg, eng in self.energy_dict.items()}
            _sorted_norm_edict = dict(sorted(_norm_edict.items()))
            return _sorted_norm_edict 
        else: 
            _norm_edict = {deg: 627.5*(eng - self.unconst_energy) for deg, eng in self.energy_dict.items()}
            _sorted_norm_edict = dict(sorted(_norm_edict.items()))
            return _sorted_norm_edict 
    
    @property
    def smile(self):
        smi_data = None
        for fn in os.listdir(self.smi_dir):
            fpath = os.path.join(self.smi_dir, fn)
            if fn.startswith(self.name[:12]): 
                with open(fpath, 'r') as smi_fn: 
                    smi_data = smi_fn.read()
                return smi_data
                break
        if smi_data is None:
            return "No smiles found for this molecule."
    
    def draw_structure(self, out_dir=None):
        molecule = Chem.MolFromSmiles(self.smile)
        AllChem.Compute2DCoords(molecule)
        if out_dir is not None: 
            Draw.MolToFile(molecule, out_dir+'img_{}.png'.format(self.name))
        return Draw.MolToImage(molecule)
    
    def plot_pe_curve(self, title, out_dir):
        """
        Plots the energy vs the torsion angle for a molecule given a torsion energy
        file. Saves the plot to a png file.
        """
        if self.unconst_data is None:
            print("No unconstrained dictionary found for {}.".format(self.name))
        phi, energy = self.norm_energy_dict.keys(), self.norm_energy_dict.values()
        min_energy, max_energy = min(energy), max(energy)

        fig, ax = plt.subplots()
        ax.set_xlim(0, 3)
        ax.set_ylim(0, 3)
        
        plt.scatter(phi, energy)
        plt.xlim(min(phi)-3, max(phi)+3)
        plt.xticks(np.linspace(start=0, stop=180, num=7))
        plt.ylim(top = 15, bottom = -1)
        plt.yticks(np.linspace(start=0, stop=14, num=8))
        plt.xlabel("dihedral angle (degrees)")
        plt.ylabel("energy (kcal/mol)")
        plt.title(title)
        fig.set_facecolor('w')
        plt.savefig(out_dir+'torsionE_plt_{}.png'.format(title), dpi=300, bbox_inches='tight')
        
        
#         image = self.Draw()
#         image.set_size_inches(1.5, 1.5)
#         ab = AnnotationBbox(image, (10, -10))
#         fig.add_artist(image)
        plt.close('all')
    
    def get_angle_from_energy(self, energy):
        for angle, eg in self.energy_dict.items():
             if energy == eg:
                return angle
        return "Dihedral angle doesn't exist for {}.".format(energy)
    
    @staticmethod
    def write_json(data, filename):
        with open(filename,'w') as f:
            json.dump(data, f, indent=2)

    
    def write_nnff_json(self, json_outdir): 
        for angle in self.norm_energy_dict.keys(): 
            json_data = {
                "molecule_name" : self.name,
                "degree" : angle,
                "smile" : self.smile,
                "energy" : self.norm_energy_dict[angle]
            }
            self.write_json(json_data, "{}/{}_{}deg.json".format(json_outdir,self.name,int(angle)))
